## Importing some stuff

In [1]:
import simpy
import random
import gym
import numpy as np
import pandas as pd

### Flow shop environment

In [2]:
class flowshopGym:
    import random
    # --------------------------------------INITIALIZE THINGS------------------------------------------
    def __init__(self):
        self.NUM_PRODUCTS = 20
        self.IAT = 1 # change this to sample from some distribution
        self.action_space = gym.spaces.Discrete(2)
        #self.observation_space = spaces.Box(-high, high, dtype=np.float32)
        self.prod_and_requests = []
        self.reward = 0 
        self.cumulative_reward = 0
        
        self.state_format = 'Hosp'
    # --------------------------------------DEFINE PROCESSES------------------------------------------
    
    # Process 1 - Spawn of products
    def _newProduct(self):
        print("Entered new product process")
        # Produce X number of products
        for i in range(self.NUM_PRODUCTS):
            
            # Creating instances of the product 
            self.instances.append(self.product(self,self.envSimpy,'Product_%d' % i,i/100, self.machine))
            #print("Product " + str(i) + "created")
            # call the Request machine process
            #self.envSimpy.process(self._requestMachine()) # removing this because this is being done inside instances
            # IAT TIMEOUT BEFORE THE NEXT SPAWN
            #next_admission = random.expovariate(1 / interarrival_time)
            yield self.envSimpy.timeout(self.IAT)
    

    
    # --------------------------------------DEFINE FUNCTIONS------------------------------------------
    
    def getObservation(self):
        # Calculate the states 
        states = []
        if self.machine.queue:
            for i in (self.machine.queue):
                states.append(((self.relater(i)).number)*100)
        
        #Checking the length of the states and filling the rest with zeros
        if len(states) < self.NUM_PRODUCTS:
            for i in range(self.NUM_PRODUCTS - len(states)):
                states.append(0)
        states = np.array(states).reshape(self.NUM_PRODUCTS,1)
        
        states = states.flatten()
        print("My states ----- ",states)
        return states
    
    def getObs(self): # delete the getObservation function once this function is properly defined. 

            
        states = dict()
        #states['current_time'] = self.envSimpy.now # current simulation time

        # Getting details of the product in machine queue
        if sim.machine.queue != []: # if the machine queue is not empty
            prod_req_1 = self.relater(sim.machine.queue[0]) # first product in the queue
            #states['MC1_queue1_due'] = prod_req_1.production_end # Due date of the first product in the queue at machine 1
            MC1_queue1_due = prod_req_1.production_end # Due date of the first product in the queue at machine 1

            states['MC1_queue1_type'] = prod_req_1.prod_type # Product type of the first product in the queue at machine 1
            states['time left'] = MC1_queue1_due - self.envSimpy.now # +ve = good / -ve = bad
        else:
            #states['MC1_queue1_due'] = None
            states['MC1_queue1_type'] = None 
            states['time left'] = None

        # Getting details of the product inside the machine
        if sim.machine.users != []: #if there is someone using the machine
            states['MC1_prod_type'] = (self.relater(sim.machine.users[0])).prod_type # Product type of the product inside machine 1
        else:
            states['MC1_prod_type'] = None


        
        
            
            
        obs = [v for k,v in states.items()]    
        print("------ obs ", states)
        
        if self.state_format == 'Hosp':
            return obs
        elif self.state_format =='OpenAI':
            return self.encode(obs)
    
    def encode(self, obs):
        
        return encoded_obs
    
    def doneFn(self):
        # So the condition is, 
        #(no items in queue1 + queue2 + ...) AND 
        #(no machine is running)
        
        if self.machine.queue != None:
            lenQueue = len(self.machine.queue)
        else: 
            lenQueue = 0 
        lenMachinesUsed = self.machine.count
        #print("Queue length",lenQueue, "no machines used", lenMachinesUsed)
        if lenQueue == 0 and lenMachinesUsed == 0: 
            #print("length of queue ",lenQueue,"Machines used ", lenMachinesUsed)
            return True
        else:
            #print("length of queue ",lenQueue,"Machines used ", lenMachinesUsed)
            return False
    
    # A function to relate products and requests
    def relater(self,item):
        # Note that the input to this function shouldn't be a list. It should be of 
        # type request or product
        output_item = None
        if item == type(list):
            print("Error passed in a list. Expecting request or product object")
            return None
        # If the input is a request
        if type(item) == simpy.resources.resource.Request:
            for i, j in enumerate(self.prod_and_requests):
                if j[1] == item:
                    output_item = j[0]
                    return output_item
        elif type(item) == simpy.resources.resource.Request: #if the input is a product 
            for i, j in enumerate(self.prod_and_requests):
                if j[0] == item:
                    output_item = j[1]
                    return output_item   
    
    
    # --------------------------------------RENDER------------------------------------------
    
    
    
    
    # --------------------------------------RESET------------------------------------------
    def reset(self):
        print("Entered Reset")
        # Initialise simpy environemnt and a machine
        self.envSimpy = simpy.Environment()
        self.machine = simpy.Resource(self.envSimpy,capacity = 1)
        
        # Initial variable definitions
        self.time_start = self.envSimpy.now
        self.next_time_stop = self.time_start + 1
        self.time_step = 0.99 
        self.time_step_terminal = self.time_start + 100
        self.current_action = 0
        self.instances = []
         
        # Set up starting processes
        self.envSimpy.process(self._newProduct())
        
        # Set starting state values

        
        # Inital load of patients (to average occupancy)
        #self._load_patients()
        
        # Starting values of observations
        
        
        # Put state dictionary items into observations list (Define observations)
        #observations = [v for k,v in self.state.items()]
        #return observations
    
    
    
    
    # --------------------------------------STEP------------------------------------------
    
    def step(self, action):
        
        # Define params
        self.current_action = action
        self.action_taken = False 
        self.info = {}
        self.reward = 0 
        #print("machine is used by",sim.machine.users)
        # Run for time_step
        self.next_time_stop += self.time_step
        self.envSimpy.run(until = self.next_time_stop)
    
        # Get values
        observation = self.getObs()
        done = self.doneFn()
        info = self.info
        self.cumulative_reward += self.reward
        # Return values
        return (observation, sim.reward, done, info)
    
    
    class product(object):
        def __init__(self, flowshopGym, envSimpy, name, number, machine):
            
            # Creating the required resources and environment
            self.envSimpy = envSimpy

            # State space variables
            self.prod_type = random.randint(0,1) # the product type of the product.
            self.production_end = random.randint(20,90)
            
            # Rest of the variables
            self.prod_state = 1 # the state of production. 1 implies raw material. 2 implies first operation done and so on. 
            self.production_time = random.randint(1,8)  
            self.name = name
            self.number = number
            self.machine = machine 
            #self.production()
            self.envSimpy.process(self.production())
            
            #self.action_taken_event = envSimpy.event
            # Defining Events        
            #self.action_accepted_event = envSimpy.event()            
            # Defining processes
            #self.check_accepted_proc = envSimpy.process(self.check_accepted(envSimpy))


            
        def production(self):
            # the first step in production is requesting for a machine
            #print('entered production ' + self.name)
            with self.machine.request() as req:
                
                sim.prod_and_requests.append([self,req])
                #print('waiting for the request to succeed ' + str(self.name))
                yield req
                print('Start production of ' + str(self.name) + ' at '+ str(self.envSimpy.now)  )
                #print("Request of the product ", envGym.relater(self))
                print('Production time is ' + str(self.production_time))
                yield self.envSimpy.timeout(self.production_time)
                self.tardiness = self.envSimpy.now - self.production_end   # -ve is good; +ve is bad 
                
                # End of production 
                print('End production of ' + str(self.name) + ' at ' + str(self.envSimpy.now))
                
                
                # Wait for an acceptance only when there are products in the machine.queue
                if (sim.machine.queue) != []:
                    # Pass time when the action chosen by the agent is rejection
                    i = 0
                    while True:
                        if i == 0:
                            yield self.envSimpy.timeout(1)
                        i += 1

                        #print("Inside current_action", sim.current_action)

                        if sim.current_action == 1: #rejected
                            
                            # rearrange the machine queue to reflect the rejection
                            sim.machine.queue.insert(len(sim.machine.queue), sim.machine.queue.pop(0))
                            print("Action rejected; Passing time by 1 ", i)
                            print("Current time " + str(self.envSimpy.now))
                            yield self.envSimpy.timeout(1)




                            # rearrange the machine queue to reflect the rejection
                            #self.machine.queue.insert(len(self.machine.queue), self.machine.queue.pop(0))


                            # print the rearranged machine queue
                            #print('#observation_after')
                            #print(envGym.getObservation())



                        else: #if action accepted
                            print("Action accepted")
                            break
                else:
                    pass
                    
                
                # Decide the reward 
                if self.tardiness < 0:
                    sim.reward = 10 
                else: 
                    sim.reward = 0
                print("Tardiness is ",self.tardiness)
                # Production has ended now I have to take the next action using take_action()
                # envGym.machine.queue = envGym.take_action()
                #envGym.machine.queue = []
        

### Overall flow

In [3]:
sim = flowshopGym()
sim.reset()
while True: # This while is for one full cycle of products (20 nos, currently)

    # Choosing the action
    #action = envGym.action_space.sample() # Random agent
    # action = 0 #FIFO
    #action = random.randint(0,1)
    action = sim.action_space.sample()
    print('Action space from the gym ', action)
    # Taking a step
    print("---------------New Cycle----------------")
    #print("Action from the agent " + str(action))
    obs, reward, done, info = sim.step(action) # the env here is gym
    if done:
        print( "End of simulation at " ,sim.envSimpy.now)
        print("Total cumulative reward ", sim.cumulative_reward)
        break
    print("Reward achieved is ", reward)

    #print("Action chosen by the agent ", action)
    #print("Action taken by the environment ", )
     

Entered Reset
Action space from the gym  1
---------------New Cycle----------------
Entered new product process
Start production of Product_0 at 0
Production time is 2
------ obs  {'MC1_queue1_type': 0, 'time left': 30.01, 'MC1_prod_type': 1}
Reward achieved is  0
Action space from the gym  0
---------------New Cycle----------------
End production of Product_0 at 2
------ obs  {'MC1_queue1_type': 0, 'time left': 29.02, 'MC1_prod_type': 1}
Reward achieved is  0
Action space from the gym  0
---------------New Cycle----------------
Action accepted
Tardiness is  -26
Start production of Product_1 at 3
Production time is 3
------ obs  {'MC1_queue1_type': 0, 'time left': 70.03, 'MC1_prod_type': 0}
Reward achieved is  10
Action space from the gym  0
---------------New Cycle----------------
------ obs  {'MC1_queue1_type': 0, 'time left': 69.04, 'MC1_prod_type': 0}
Reward achieved is  0
Action space from the gym  0
---------------New Cycle----------------
------ obs  {'MC1_queue1_type': 0, 'time

In [4]:
np.floor(-41.72999999999972)

-42.0